In [1]:
# Notebook: 03_merge_and_standardize.ipynb
# Purpose: WBGT（気候データ）と WDI（エネルギー構造＋社会経済データ）を年次パネル形式で結合する

# --------------------------------------------------
# ライブラリのインポート
# --------------------------------------------------
import pandas as pd
import glob
import os

In [2]:
# --------------------------------------------------
# ファイルパスとマッピング設定
# --------------------------------------------------

# WBGT データが保存されているディレクトリ
wbgt_dir = "../data/processed/wbgt/"

# WDI データのファイルパス
wdi_path = "../data/processed/wdi/wdi_southasia_2000_2023.csv"

# 南アジア 6ヵ国の ISO3 → 国名 マッピング
iso3_to_country = {
    "IND": "India",
    "BGD": "Bangladesh",
    "PAK": "Pakistan",
    "NPL": "Nepal",
    "LKA": "Sri Lanka",
    "BTN": "Bhutan"
}

In [3]:
# --------------------------------------------------
# WBGT CSV をすべて読み込み、年次パネル用 DataFrame を作成
# --------------------------------------------------

# WBGT データの CSV パターン
wbgt_pattern = os.path.join(wbgt_dir, "wbgt_by_country_*.csv")
wbgt_files = glob.glob(wbgt_pattern)

dfs_wbgt = []

for filepath in wbgt_files:
    # ファイル名から年を抽出（例：「.../wbgt_by_country_2020.csv」 → 「2020」）
    basename = os.path.basename(filepath)
    # 'wbgt_by_country_YYYY.csv' の形式を想定
    year_str = basename.replace("wbgt_by_country_", "").replace(".csv", "")
    year = int(year_str)
    
    # CSV を読み込み
    df = pd.read_csv(filepath)  # columns: ['country', 'mean_wbgt']
    
    # 年情報を列として追加
    df["year"] = year
    
    # データをリストに追加
    dfs_wbgt.append(df)

# すべてを縦に結合して 1 DataFrame に
df_wbgt = pd.concat(dfs_wbgt, ignore_index=True)

# 確認
print("WBGT DataFrame サンプル:")
print(df_wbgt.head())
print("\nUnique years in WBGT:", df_wbgt["year"].unique())
print("Unique countries in WBGT:", df_wbgt["country"].unique())

WBGT DataFrame サンプル:
      country  mean_wbgt  year
0  Bangladesh  22.917103  2000
1      Bhutan   6.482861  2000
2       India  19.745287  2000
3       Nepal  10.267117  2000
4    Pakistan  17.338387  2000

Unique years in WBGT: [2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023]
Unique countries in WBGT: ['Bangladesh' 'Bhutan' 'India' 'Nepal' 'Pakistan' 'Sri Lanka']


In [4]:
# --------------------------------------------------
# WDI データの読み込みと国名列の追加
# --------------------------------------------------

# WDI CSV を読み込み
df_wdi = pd.read_csv(wdi_path)
# 現在の df_wdi のカラム（例: ['iso3', 'year', 'renewable_energy_pct', … ]）
print("\nWDI DataFrame columns:", df_wdi.columns.tolist())

# ISO3 → 国名 を新たに 'country' 列として追加
df_wdi["country"] = df_wdi["iso3"].map(iso3_to_country)

# 確認：欠損がないか
missing_countries = df_wdi["country"].isna().sum()
print(f"\nWDI DataFrame で 'country' が NaN になっている行数: {missing_countries}")

print("\nWDI DataFrame サンプル:")
print(df_wdi.head())


WDI DataFrame columns: ['iso3', 'year', 'renewable_energy_pct', 'fossil_fuel_pct', 'electricity_per_capita', 'co2_per_capita', 'gdp_per_capita', 'unemployment_rate', 'health_expenditure_pct', 'agri_valueadded_pct', 'urbanization_pct']

WDI DataFrame で 'country' が NaN になっている行数: 0

WDI DataFrame サンプル:
  iso3  year  renewable_energy_pct  fossil_fuel_pct  electricity_per_capita  \
0  BGD  2000                  60.2            57.93               99.253551   
1  BGD  2001                  55.9            61.28              109.490964   
2  BGD  2002                  54.4            61.25              117.271799   
3  BGD  2003                  52.6            62.48              123.721051   
4  BGD  2004                  52.0            62.76              158.478288   

   co2_per_capita  gdp_per_capita  unemployment_rate  health_expenditure_pct  \
0        0.200483      396.670730              3.270                1.805045   
1        0.238508      394.656410              3.610           

In [5]:
# --------------------------------------------------
# WBGT と WDI のマージ（キー: country + year）
# --------------------------------------------------

# 内部結合 (inner) でマージする。欠損行を除去したい場合は this; 欠損も残したい場合は how="left"
df_panel = pd.merge(
    df_wbgt,
    df_wdi,
    on=["country", "year"],
    how="inner"
)

# 確認
print("\ndf_panel サンプル (マージ結果):")
print(df_panel.head())
print("\n結合後の行数:", len(df_panel))


df_panel サンプル (マージ結果):
      country  mean_wbgt  year iso3  renewable_energy_pct  fossil_fuel_pct  \
0  Bangladesh  22.917103  2000  BGD                  60.2            57.93   
1      Bhutan   6.482861  2000  BTN                  91.4              NaN   
2       India  19.745287  2000  IND                  46.9            67.24   
3       Nepal  10.267117  2000  NPL                  88.3            11.97   
4    Pakistan  17.338387  2000  PAK                  51.0            58.90   

   electricity_per_capita  co2_per_capita  gdp_per_capita  unemployment_rate  \
0               99.253551        0.200483      396.670730              3.270   
1                     NaN        0.682234      771.924911              1.660   
2              384.988419        0.941139      442.750219              7.624   
3               57.360499        0.139299      223.830289             10.551   
4              333.072642        0.722281      642.338346              0.634   

   health_expenditure_pct 

In [6]:
# --------------------------------------------------
# 欄の並び替えと保存
# --------------------------------------------------

# 必要に応じて列の順序を整理
# ここでは例えば 'country', 'iso3', 'year', 'mean_wbgt', それ以外の WDI 指標
cols_order = [
    "country", "iso3", "year", "mean_wbgt",
    "renewable_energy_pct", "fossil_fuel_pct", "electricity_per_capita", "co2_per_capita",
    "gdp_per_capita", "unemployment_rate", "health_expenditure_pct", "agri_valueadded_pct", "urbanization_pct"
]
# 存在しない列があれば表示しておく
missing_cols = [c for c in cols_order if c not in df_panel.columns]
print(f"\n指定した order に存在しない列（あれば）: {missing_cols}")

# 実際に存在するカラムだけ抽出して順序を整える
cols_existing = [c for c in cols_order if c in df_panel.columns]
df_panel = df_panel[cols_existing]

# 保存ディレクトリを準備
os.makedirs("../data/processed/panel", exist_ok=True)

# スケール前のパネルデータを保存（標準化前）
df_panel.to_csv("../data/processed/panel/panel_southasia_2000_2023.csv", index=False)
print("\npanel_southasia_2000_2023.csv を保存しました。")


指定した order に存在しない列（あれば）: []

panel_southasia_2000_2023.csv を保存しました。


In [7]:
# --------------------------------------------------
# 欠損チェック（オプション）
# --------------------------------------------------

# 欠損数の確認
print("\npanel データ欠損数確認:")
print(df_panel.isna().sum())

# 欠損が多い列があればコメントする
na_counts = df_panel.isna().sum()
print("\n欠損の多い列（欠損数 > 0 のもの）:")
print(na_counts[na_counts > 0])


panel データ欠損数確認:
country                    0
iso3                       0
year                       0
mean_wbgt                  0
renewable_energy_pct      11
fossil_fuel_pct           25
electricity_per_capita    29
co2_per_capita             0
gdp_per_capita             1
unemployment_rate          0
health_expenditure_pct     6
agri_valueadded_pct        1
urbanization_pct           0
dtype: int64

欠損の多い列（欠損数 > 0 のもの）:
renewable_energy_pct      11
fossil_fuel_pct           25
electricity_per_capita    29
gdp_per_capita             1
health_expenditure_pct     6
agri_valueadded_pct        1
dtype: int64


In [19]:
### Bhutan と 2022/2023 年を除外する
# 例: df_panel の先頭数行を確認しておく
print("フィルタ前 df_panel のサンプル:")
print(df_panel.head(), "\n")
print("フィルタ前の行数:", len(df_panel))

# --------------------------------------------------
# 1. 除外条件を定義
# --------------------------------------------------

# 国名列が 'country'、年列が 'year' である前提
# Bhutan を除外
cond_not_bhutan = df_panel["country"] != "Bhutan"

# 年が 2021 以下（すなわち 2022, 2023 を除外）
cond_year_upto_2021 = df_panel["year"] <= 2021

# (両方の条件を満たす行のみを残す)
cond_keep = cond_not_bhutan & cond_year_upto_2021

# --------------------------------------------------
# 2. データをフィルタリング
# --------------------------------------------------

df_filtered = df_panel[cond_keep].reset_index(drop=True)

# 確認: 何行削られたか
print("フィルタ後 df_filtered のサンプル:")
print(df_filtered.head(), "\n")
print("フィルタ後の行数:", len(df_filtered))
print("含まれている国一覧:", df_filtered["country"].unique())
print("含まれている年一覧:", df_filtered["year"].unique())


フィルタ前 df_panel のサンプル:
      country iso3  year  mean_wbgt  renewable_energy_pct  fossil_fuel_pct  \
0  Bangladesh  BGD  2000  22.917103                  60.2            57.93   
1      Bhutan  BTN  2000   6.482861                  91.4              NaN   
2       India  IND  2000  19.745287                  46.9            67.24   
3       Nepal  NPL  2000  10.267117                  88.3            11.97   
4    Pakistan  PAK  2000  17.338387                  51.0            58.90   

   electricity_per_capita  co2_per_capita  gdp_per_capita  unemployment_rate  \
0               99.253551        0.200483      396.670730              3.270   
1                     NaN        0.682234      771.924911              1.660   
2              384.988419        0.941139      442.750219              7.624   
3               57.360499        0.139299      223.830289             10.551   
4              333.072642        0.722281      642.338346              0.634   

   health_expenditure_pct  a

In [20]:
# 例: 既に定義済みの数値変数リスト
numeric_cols_list = [
    "mean_wbgt",
    "renewable_energy_pct",
    "fossil_fuel_pct",
    "electricity_per_capita",
    "co2_per_capita",
    "gdp_per_capita",
    "unemployment_rate",
    "health_expenditure_pct",
    "agri_valueadded_pct",
    "urbanization_pct"
]

# 新たに df_imputed をフィルタ後のデータで作成
df_imputed = df_filtered.sort_values(["country", "year"]).reset_index(drop=True)

# グループ単位で線形補間＋ffill/bfill
def interpolate_group(grp):
    grp_interp = grp.interpolate(method="linear", limit_direction="both")
    grp_filled = grp_interp.ffill().bfill()
    return grp_filled

df_imputed[numeric_cols_list] = (
    df_imputed
    .groupby("country")[numeric_cols_list]
    .transform(lambda grp: interpolate_group(grp))
)

# 補間後の欠損チェック
print("\n補間後の欠損数（各列ごと）:")
print(df_imputed[numeric_cols_list].isna().sum())



補間後の欠損数（各列ごと）:
mean_wbgt                 0
renewable_energy_pct      0
fossil_fuel_pct           0
electricity_per_capita    0
co2_per_capita            0
gdp_per_capita            0
unemployment_rate         0
health_expenditure_pct    0
agri_valueadded_pct       0
urbanization_pct          0
dtype: int64


In [22]:
from sklearn.preprocessing import StandardScaler

df_panel_scaled = df_imputed.copy()
scaler = StandardScaler()

numeric_cols_list = [
    "mean_wbgt",
    "renewable_energy_pct",
    "fossil_fuel_pct",
    "electricity_per_capita",
    "co2_per_capita",
    "gdp_per_capita",
    "unemployment_rate",
    "health_expenditure_pct",
    "agri_valueadded_pct",
    "urbanization_pct"
]

# 数値列をまとめて一度に fit/transform
df_panel_scaled[numeric_cols_list] = scaler.fit_transform(df_panel_scaled[numeric_cols_list])

# 標準化後のサンプルを表示
print("\n標準化後サンプル:")
print(df_panel_scaled[numeric_cols_list].head())



標準化後サンプル:
   mean_wbgt  renewable_energy_pct  fossil_fuel_pct  electricity_per_capita  \
0   0.723763              0.374562         0.173344               -1.317474   
1   0.753233              0.138117         0.327887               -1.272254   
2   0.769910              0.055637         0.326503               -1.237885   
3   0.766351             -0.043340         0.383246               -1.209398   
4   0.770875             -0.076332         0.396163               -1.055871   

   co2_per_capita  gdp_per_capita  unemployment_rate  health_expenditure_pct  \
0       -1.214646       -0.928260          -0.880881               -1.651258   
1       -1.128869       -0.930236          -0.774189               -1.688075   
2       -1.109330       -0.930992          -0.662477               -1.507313   
3       -1.091101       -0.898748          -0.551392               -1.476823   
4       -1.068581       -0.870427          -0.553274               -1.435553   

   agri_valueadded_pct  urbanizat

In [23]:
os.makedirs("../data/processed/panel", exist_ok=True)
df_panel_scaled.to_csv("../data/processed/panel/panel_southasia_2000_2021_scaled.csv", index=False)
print("\npanel_southasia_2000_2021_scaled.csv を保存しました。")


panel_southasia_2000_2021_scaled.csv を保存しました。
